In [1]:
####This notebook required run on  parallel algorithms which base on MPI####
import  numpy  as  np
import libpysal as ps
from stwr.gwr import GWR, MGWR,STWR
from stwr.sel_bw import *
from stwr.utils import shift_colormap, truncate_colormap
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import pyplot
import pandas as pd
import math
from matplotlib.gridspec import GridSpec
import time
import csv 
import copy 
import tracemalloc

In [2]:
str_path = './TestDataSize 1k-10k/'

different_size_time_consuming =[]

csvfilename = str_path + 'test_data_1000.csv'
cal_coords_list =[]
cal_y_list =[]
cal_X_list =[]
delt_stwr_intervel =[0.0]

csvFile = open(csvfilename, "r")
#ReadData form the floder to test comsuming time 
df = pd.read_csv(csvFile,header = 0,names=['cal_coordsX','cal_coordsY','cal_x1','cal_x2','cal_y','time_stamp'],
                 dtype = {"cal_coordsX" : "float64","cal_coordsY":"float64",
                          "cal_x1":"float64","cal_x2":"float64",
                          "cal_y" : "float64",
                          "time_stamp":"float64"},
                 skip_blank_lines = True,
                 keep_default_na = False)
df.info()
df = df.sort_values(by=['time_stamp'])  
all_data = df.values
tick_time = all_data[0,-1]
cal_coord_tick = []
cal_X_tick =[]
cal_y_tick =[]
time_tol = 1.0e-7

lensdata = len(all_data)
for row in range(lensdata):
    cur_time = all_data[row,-1]
    if(abs(cur_time-tick_time)>time_tol):
        cal_coords_list.append(np.asarray(cal_coord_tick))
        cal_X_list.append(np.asarray(cal_X_tick))
        cal_y_list.append(np.asarray(cal_y_tick))
        delt_t = cur_time - tick_time
        delt_stwr_intervel.append(delt_t) 
        tick_time =cur_time
        cal_coord_tick = []
        cal_X_tick =[]
        cal_y_tick =[]
    coords_tick = np.array([all_data[row,0],all_data[row,1]])
    cal_coord_tick.append(coords_tick)
    x_tick = np.array([all_data[row,2],all_data[row,3]])
    cal_X_tick.append(x_tick)
    y_tick = np.array([all_data[row,4]])
    cal_y_tick.append(y_tick)
cal_cord_gwr = np.asarray(cal_coord_tick)
cal_X_gwr  = np.asarray(cal_X_tick)
cal_y_gwr = np.asarray(cal_y_tick)  
cal_coords_list.append(np.asarray(cal_coord_tick))
cal_X_list.append(np.asarray(cal_X_tick))
cal_y_list.append(np.asarray(cal_y_tick))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 199
Data columns (total 6 columns):
cal_coordsX    1000 non-null float64
cal_coordsY    1000 non-null float64
cal_x1         1000 non-null float64
cal_x2         1000 non-null float64
cal_y          1000 non-null float64
time_stamp     1000 non-null float64
dtypes: float64(6)
memory usage: 54.7 KB


In [6]:
#Test STWR memory userage
start = time.time()
tracemalloc.start()

stwr_selector_ = Sel_Spt_BW(cal_coords_list, cal_y_list, cal_X_list,#gwr_bw0,
                            delt_stwr_intervel,spherical = False)


optalpha,optsita,opt_btticks,opt_gwr_bw0 = stwr_selector_.search() 

current_used1, peak_1 = tracemalloc.get_traced_memory()    
mem_size_used1.append(current_used1)
mem_size_peaks1.append(peak_1)
stwr_model = STWR(cal_coords_list,cal_y_list,cal_X_list,delt_stwr_intervel,optsita,opt_gwr_bw0,tick_nums=opt_btticks+1,alpha =optalpha,spherical = True,recorded=1)

current_used2, peak_2 = tracemalloc.get_traced_memory()    
tracemalloc.stop()
mem_size_used2.append(current_used2)
mem_size_peaks2.append(peak_2)
end = time.time()
comsuming = end - start
different_size_time_consuming.append(comsuming)
str_comsuming = '{} seconds'.format(comsuming)
strprint ='Time-consuming of STWR when calibrating the splitting_sizes: ('+ str(sz_mem) + ' ) '+ str_comsuming
print(strprint)
stwr_results = stwr_model.fit()
print(stwr_results.summary())


Time-consuming of Fast-STWR when calibrating the splitting_sizes: (10000000000 ) 705.17955327034 seconds
Model type                                                         Gaussian
Number of observations:                                                1000
Number of covariates:                                                     3

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                        2904912.904
Log-likelihood:                                                   -1242.147
AIC:                                                               2490.295
AICc:                                                              2492.500
BIC:                                                            2903869.136
R2:                                                                   0.014
Adj. R2:                                                              0.004

Variable                       

In [7]:
print(f"Current memory usage is {mem_size_used1[0] / 10**6}MB; Peak was {mem_size_peaks1[0] / 10**6}MB")
print(f"Current memory usage is {mem_size_used2[0] / 10**6}MB; Peak was {mem_size_peaks2[0] / 10**6}MB")

Current memory usage is 9.337755MB; Peak was 16.869336MB
Current memory usage is 14.206351MB; Peak was 23.182052MB
